In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
class SoftSet:
    @staticmethod
    def clasification(X, Y, sample):
        results = []
        for i in range(0, len(X), 1):
            tmp = 0
            for k in Y[i]:
                tmp += Y[i][k] * sample[k]
            results.append(tmp)
        index = results.index(max(results))
        return X[index]
    
    @staticmethod
    def characterization(X, quantile):
        colnames = X.columns[:-1]
        names = []
        attributes = {}
        majority = int(0.4*len(X))
        i = 0
        for colname in colnames:
            attributes[colname+'<2qt'] = 0
            attributes[colname+'<3qt'] = 0
            attributes[colname+'<4qt'] = 0
            attributes[colname+'<5qt'] = 0
            names.append(colname+'<2qt')
            names.append(colname+'<3qt')
            names.append(colname+'<4qt')
            names.append(colname+'<5qt')            
        for row in X.iloc():
            i = 0
            for j in row:
                if isinstance(j, str)!=True:
                    if j < quantile.iloc[1,i]:
                        name = names[i*4]
                        attributes[name]=attributes[name]+1
                    elif j < quantile.iloc[2, i] and j > quantile.iloc[1, i]:
                        name = names[i*4+1]
                        attributes[name]=attributes[name]+1
                    elif j < quantile.iloc[3, i] and j > quantile.iloc[2, i]:
                        name = names[i*4+2]
                        attributes[name]=attributes[name]+1
                    elif j < quantile.iloc[4, i] and j > quantile.iloc[3, i]:
                        name = names[i*4+3]
                        attributes[name]=attributes[name]+1
                    i+=1
        if len(X)>=2:
            for name in names:
                if attributes[name]>=majority:
                    attributes[name]=1
                else:
                    attributes[name]=0
                
        return attributes

In [ ]:
voices = pd.read_csv('../input/voicegender/voice.csv')
female = voices.loc[voices.label=='female']
male = voices.loc[voices.label=='male']
quantile = voices.quantile([0,0.25,0.5,0.75,1])
average = voices.mean()
Y1 = SoftSet.characterization(female, quantile)
Y2 = SoftSet.characterization(male, quantile)
correct = 0
Y = [Y1, Y2]
k =200
l =30
tests = []
for j in range(l):
    correct = 0
    for i in range(1, k):
        x = random.randint(1, 3167)
        testSample = voices.iloc[x-1:x,:]
        testSample1 = SoftSet.characterization(testSample, quantile)
        check = SoftSet.clasification(['female', 'male'], Y, testSample1)
        if check == testSample.iloc[0, 20]:
            correct += 1
    accuracy=correct/k*100
    tests.append(accuracy)
    print("Accuracy for", k ,"samples:", accuracy, "%, accurate: ", correct)
overall = sum(tests)/l
print("Overall accuracy for ", l, "attempts:", overall, "%")